In [23]:
from MyDataset import TGS_Dataset
from PSPNet_models.pspnet import Res34Unetv4
import os
import torch
from torch.autograd import Variable
import pandas as pd
import numpy as np
from Evaluation import do_length_decode, do_length_encode
import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
print('load finish')

load finish


In [24]:
TRAIN_PATH = './Data/train'
train_dataset = TGS_Dataset(TRAIN_PATH)
print('finish')

1562 empty masks out of 4000 total masks
finish


In [26]:
def average_fold_prediction(path_list, H=101, W=101, fill_value=255, threshold=0.5):
    '''load rle from df, average them and return a new rle'''
    folds = []
    # decode
    for p in path_list:
        df = pd.read_csv(p)
        im = []
        for i in range(len(df)):
            im.append(do_length_decode(str(df.rle_mask.iloc[i]), H, W, fill_value))
        folds.append(im)
    # average
    avg = np.mean(folds, axis=0)
    avg = avg > threshold
    # encode
    rle = []
    for i in range(len(avg)):
        rle.append(do_length_encode(avg[i]))
    # create sub
    df = pd.DataFrame(dict(id=df.id, rle_mask=rle))
    
    return df

def load_net_and_predict(net, train_path, load_paths, batch_size=32, threshold=0.5, min_size=0):
    train_dataset = TGS_Dataset(train_path)
    pseudo_loader, pseudo_ids = train_dataset.selfdefine_dataloader_actual(flag=True, num_workers=11, batch_size=batch_size)
    # predict
    for i in range(len(load_paths)):
        net.load_state_dict(torch.load(load_paths[i]))
        p = net.predict(pseudo_loader, threshold=threshold, return_rle=False)
        if not i:
            avg = np.zeros_like(p['pred'])
        avg = (i * avg + p['pred']) / (i + 1)
    avg = avg > threshold
    # encode
    rle = []
    for i in range(len(avg)):
        rle.append(do_length_encode(avg[i]))
    # create sub
    df = pd.DataFrame(dict(id=p['id'], rle_mask=rle))
    
    return df


In [ ]:
if __name__ == '__main__':
    TRAIN_PATH = './Data/train'
    net = PSPNet()
    NET_NAME = type(net).__name__
    THRESHOLD = 0.5
    MIN_SIZE = 0
    BATCH_SIZE = 32
    
    LOAD_PATHS = [
        './Saves/Res34Unetv4/Fold0_Epoch141_Val0.612',
        './Saves/Res34Unetv4/Fold1_Epoch150_Val0.578',
        './Saves/Res34Unetv4/Fold2_Epoch150_Val0.662',
        './Saves/Res34Unetv4/Fold3_Epoch141_Val0.616',
        './Saves/Res34Unetv4/Fold4_Epoch150_Val0.589'
    ]
    
    df = load_net_and_predict(net, TRAIN_PATH, LOAD_PATHS, batch_size=BATCH_SIZE, threshold=THRESHOLD, min_size=MIN_SIZE)
    df.to_csv(os.path.join('./Saves',NET_NAME,'{}_5foldAvg.csv'.format(NET_NAME)),
        index=False)
    print('finish')

1562 empty masks out of 4000 total masks
